<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [15]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV

import nltk
from nltk.stem import WordNetLemmatizer 
  
import re
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

RANDOM_STATE=12345

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [35]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.head(), data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


(   Unnamed: 0                                               text  toxic
 0           0  Explanation\nWhy the edits made under my usern...      0
 1           1  D'aww! He matches this background colour I'm s...      0
 2           2  Hey man, I'm really not trying to edit war. It...      0
 3           3  "\nMore\nI can't make any real suggestions on ...      0
 4           4  You, sir, are my hero. Any chance you remember...      0,
 None)

In [10]:
#Посмотрим наличие пропусков

data.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

In [11]:
#Посмотрим на наличие явных дубликатов

data.duplicated().sum()

0

Так как перед нами задачи классификации, необходимо посмотреть на сбалансированность классов в целевом признаке **data['toxic']**.

In [12]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Классы не сбалансированы, перед обучением данных нужно это исправить.

In [36]:
#Лемматизируем исходный текст

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(text):
    tag = nltk.pos_tag([text])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemma_text(text):
    text = text.lower()
    lemm_text = "".join(lemmatizer.lemmatize(text, get_wordnet_pos(text)))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

data['lemm_text'] = data['text'].apply(lemma_text)

#Создадим новый датафрейм без столбца text

data_clear = data.drop(['text'], axis=1)

In [21]:
data_clear.head()

,Unnamed: 0,toxic,lemm_text
0,0,0,explanation why the edits made under my userna...
1,1,0,d aww he matches this background colour i m se...
2,2,0,hey man i m really not trying to edit war it s...
3,3,0,more i can t make any real suggestions on impr...
4,4,0,you sir are my hero any chance you remember wh...


Разобьем данные на выборки, а затем напишем функцию upsample для балансировки классов.

In [56]:
features = data_clear['lemm_text']
target = data_clear['toxic']

features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              train_size=0.7, 
                                                                              random_state=RANDOM_STATE)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            test_size=0.5,
                                                                            random_state=RANDOM_STATE)

In [57]:
#Напишем функцию upsample для балансировки классов

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=RANDOM_STATE)
    
    return features_upsampled, target_upsampled

In [58]:
#Сбалансируем данные

features_upsampled, target_upsampled = upsample(features_train, target_train, 9)

In [26]:
#Проверим балансировку

target_upsampled.value_counts()

1    73125
0    71521
Name: toxic, dtype: int64

In [59]:
#Преобразуем текст в TF-IDF векторы для моделей

stopwordss = set(stopwords.words('english'))

count_tf = TfidfVectorizer(stop_words = stopwordss, use_idf=True)

features_train = count_tf.fit_transform(features_upsampled)
features_valid = count_tf.transform(features_valid)
features_test = count_tf.transform(features_test)

# Вывод

Загрузили данные и подготовили их для обучения моделей. 

Данные содержат 2 столбца:

**text** — текст комментариев;

**toxic** — хранит информацию о комментариях: токсичный (1) и положительный (0) комментарии.

Пропусков и явных дубликатов не обнаружено, в целевом признаке классы не сбалансированы.

Лемматизировали исходный текст. Разбили данные на 2 выборки: обучающую и тестовую. Произвели балансировку классов, а затем преобразовали текст в TF-IDF векторы.

## Обучение

# LogisticRegression

In [13]:
#Подберем параметры для модели 

#model_lr = LogisticRegression()

#param = {'C':[10], 'random_state':[RANDOM_STATE]}

#model_lr_grid = GridSearchCV(model_lr, param, scoring='f1', cv=3)
#model_lr_grid.fit(features_train, target_upsampled)

#print('F1:', model_lr_grid.best_score_)

In [61]:
#Обучим модель и вычислим значение метрики f1

model_lr = LogisticRegression(C=10, random_state=RANDOM_STATE)
model_lr.fit(features_train, target_upsampled)

prediction = model_lr.predict(features_valid)

print('F1:', f1_score(target_valid, prediction))

F1: 0.7670866743031692


# RandomForestClassifier

In [15]:
#Подберем параметры для модели 

#model_rf = RandomForestClassifier()

#param = {'n_estimators': range(1,50,5),'max_depth': range(1,15,3)}

#model_rf_grid = GridSearchCV(model_rf, param, scoring=r'f1', cv=3)
#model_rf_grid.fit(features_train, target_upsampled)

#print(model_rf_grid.best_params_)

#print('F1:', model_rf_grid.best_score_)


In [62]:
#Обучим модель и вычислим значение метрики f1

model_rf = RandomForestClassifier(max_depth=13, n_estimators=46, random_state=RANDOM_STATE)
model_rf.fit(features_train, target_upsampled)

prediction = model_rf.predict(features_valid)

print('F1:', f1_score(target_valid, prediction))

F1: 0.35199870529211846


# LGBMClassifier

In [64]:
#Обучим модель и вычислим значение метрики f1

model_lgbm = LGBMClassifier(num_leaves=100, learning_rate=0.1, random_state=RANDOM_STATE)
model_lgbm.fit(features_train, target_upsampled)

prediction = model_lgbm.predict(features_valid)

print('F1:', f1_score(target_valid, prediction))

F1: 0.7479733381372726


# CatBoostClassifier

In [63]:
#Обучим модель и вычислим значение метрики f1

model_cat = CatBoostClassifier(depth=6, iterations=60, learning_rate=0.9)
model_cat.fit(features_train, target_upsampled)

prediction = model_cat.predict(features_valid)

print('F1:', f1_score(target_valid, prediction))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.5435533	total: 3.97s	remaining: 3m 54s
1:	learn: 0.4938819	total: 7.19s	remaining: 3m 28s
2:	learn: 0.4608281	total: 10.5s	remaining: 3m 19s
3:	learn: 0.4376219	total: 13.8s	remaining: 3m 13s
4:	learn: 0.4192844	total: 17.1s	remaining: 3m 7s
5:	learn: 0.4041569	total: 20.4s	remaining: 3m 3s
6:	learn: 0.3921997	total: 23.6s	remaining: 2m 58s
7:	learn: 0.3807499	total: 26.9s	remaining: 2m 54s
8:	learn: 0.3708438	total: 30.1s	remaining: 2m 50s
9:	learn: 0.3626015	total: 33.3s	remaining: 2m 46s
10:	learn: 0.3558703	total: 36.6s	remaining: 2m 42s
11:	learn: 0.3497236	total: 39.7s	remaining: 2m 38s
12:	learn: 0.3445164	total: 42.9s	remaining: 2m 35s
13:	learn: 0.3394178	total: 46s	remaining: 2m 31s
14:	learn: 0.3346800	total: 49s	remaining: 2m 26s
15:	learn: 0.3301049	total: 52.2s	remaining: 2m 23s
16:	learn: 0.3241810	total: 55.6s	remaining: 2m 20s
17:	learn: 0.3195453	total: 58.8s	remaining: 2m 17s
18:	learn: 0.3151646	total: 1m 1s	remaining: 2m 13s
19:	learn: 0.3113449	total: 

Моделям LGBMClassifier и CatBoostClassifier я также пыталась подобрать гиперпараметры, но слетало с dead kernel, поэтому взяла значения из прошлого проекта.

In [65]:
#Выведем таблицу с результатами моделей

data_tabl=[0.7670, 0.3519, 0.7479, 0.699]
model_tabl=['LogisticRegression', 'RandomForestClassifier', 'LGBMClassifier', 'CatBoostClassifier']

pd.DataFrame(data=data_tabl, index=model_tabl, columns=['Метрика f1'])

,Метрика f1
LogisticRegression,0.7670
RandomForestClassifier,0.3519
LGBMClassifier,0.7479
CatBoostClassifier,0.6990


Лучшее значение метрики f1 (0.7670) у модели LogisticRegression. Посмотрим результат на тестовой выборке.

In [66]:
#Вычислим значение метрики f1 на тестовой выборке

prediction = model_lr.predict(features_test)

print('F1:', f1_score(target_test, prediction))

F1: 0.7611881188118812


In [67]:
#Вычислим f1 констатной модели

model_const = DummyClassifier(strategy="most_frequent")
model_const.fit(features_train, target_upsampled)

prediction = model_const.predict(features_test)

print('F1:', f1_score(target_test,prediction))

F1: 0.18324209245742093


## Выводы

По условию задачи нужно было построить модель со значением метрики качества F1 не меньше 0.75. Мы рассмотрели следующие модели:

1. LogisticRegression;

2. RandomForestClassifier;

3. LGBMClassifier;

4. CatBoostClassifier.

Мы достигли метрики f1=0.7611 на тестовой выборке у модели LogisticRegression. В сравнении с константной моделью, она адекватна. Ближе всего к установленному порогу оказалась модель LGBMClassifier, на валидационной выборке её метрика f1=0.7479. Эта модель, как и остальные, в сравнении с константной моделью оказалась адекватной. Худший результат показала модель RandomForestClassifier со значением 0.3519.